In [1]:
import sympy as sy
import re
from sympy.codegen.ast import real, float80
from sympy.printing import ccode, fcode
sy.init_printing()

In [14]:
b = sy.symbols(['x_bjg','y_bjg','z_bjg', 'w_bjg', 'phi_bjg', 'K_bjg'])
ins = sy.symbols(['x_Sjb','y_Sjb','z_Sjb', 'w_Sb', 'phi_Sb', 'K_Sb'])
pt = sy.symbols(['x_sj', 'y_sj', 'z_sj'])
plane = sy.symbols(['n_xpg', 'n_ypg', 'n_zpg', 'n_dpg'])

# Make the Rotation matrix

In [15]:
R_x = sy.Matrix([[1,0,0],
               [0, sy.cos(ins[3]), -1*sy.sin(ins[3])], 
               [0, sy.sin(ins[3]), sy.cos(ins[3])]])

R_y = sy.Matrix([[sy.cos(ins[4]),0,sy.sin(ins[4])],
               [0, 1, 0], 
               [-1*sy.sin(ins[4]), 0, sy.cos(ins[4])]])

R_z = sy.Matrix([[sy.cos(ins[5]),-1*sy.sin(ins[5]),0],
               [sy.sin(ins[5]), sy.cos(ins[5]), 0], 
               [0, 0, 1]])

In [16]:
R_x

⎡1      0          0     ⎤
⎢                        ⎥
⎢0  cos(w_Sb)  -sin(w_Sb)⎥
⎢                        ⎥
⎣0  sin(w_Sb)  cos(w_Sb) ⎦

In [17]:
R_y

⎡cos(φ_Sb)   0  sin(φ_Sb)⎤
⎢                        ⎥
⎢    0       1      0    ⎥
⎢                        ⎥
⎣-sin(φ_Sb)  0  cos(φ_Sb)⎦

In [18]:
R_z

⎡cos(K_Sb)  -sin(K_Sb)  0⎤
⎢                        ⎥
⎢sin(K_Sb)  cos(K_Sb)   0⎥
⎢                        ⎥
⎣    0          0       1⎦

In [19]:
R = R_z*R_y*R_x

In [20]:
R

⎡cos(K_Sb)⋅cos(φ_Sb)  -sin(K_Sb)⋅cos(w_Sb) + sin(φ_Sb)⋅sin(w_Sb)⋅cos(K_Sb)  si
⎢                                                                             
⎢sin(K_Sb)⋅cos(φ_Sb)  sin(K_Sb)⋅sin(φ_Sb)⋅sin(w_Sb) + cos(K_Sb)⋅cos(w_Sb)   si
⎢                                                                             
⎣    -sin(φ_Sb)                       sin(w_Sb)⋅cos(φ_Sb)                     

n(K_Sb)⋅sin(w_Sb) + sin(φ_Sb)⋅cos(K_Sb)⋅cos(w_Sb)⎤
                                                 ⎥
n(K_Sb)⋅sin(φ_Sb)⋅cos(w_Sb) - sin(w_Sb)⋅cos(K_Sb)⎥
                                                 ⎥
              cos(φ_Sb)⋅cos(w_Sb)                ⎦

In [21]:
R_x = sy.Matrix([[1,0,0],
               [0, sy.cos(b[3]), -1*sy.sin(b[3])], 
               [0, sy.sin(b[3]), sy.cos(b[3])]])

R_y = sy.Matrix([[sy.cos(b[4]),0,sy.sin(b[4])],
               [0, 1, 0], 
               [-1*sy.sin(b[4]), 0, sy.cos(b[4])]])

R_z = sy.Matrix([[sy.cos(b[5]),-1*sy.sin(b[5]),0],
               [sy.sin(b[5]), sy.cos(b[5]), 0], 
               [0, 0, 1]])
R_b =  R_z*R_y*R_x

# Make the vectors

In [22]:
r_eop = sy.Matrix([ins[0],ins[1],ins[2]])
r_b = sy.Matrix([b[0],b[1],b[2]])
r_pt = sy.Matrix([pt[0],pt[1],pt[2]])
r_plane = sy.Matrix([plane[0],plane[1],plane[2]])

In [23]:
fx_1 = r_eop + R*r_b + R*R_b*r_pt
fx = fx_1.dot(r_plane) + plane[3]

# Do the Derivitives

In [24]:
# d/d b
d_fx_bx = sy.Derivative(fx,b[0] ).doit()
d_fx_by = sy.Derivative(fx,b[1] ).doit()
d_fx_bz = sy.Derivative(fx,b[2] ).doit()

d_fx_bw = sy.Derivative(fx,b[3] ).doit()
d_fx_bp = sy.Derivative(fx,b[4] ).doit()
d_fx_bk = sy.Derivative(fx,b[5] ).doit()

# d/d plane
d_fx_a = sy.Derivative(fx,plane[0] ).doit()
d_fx_b = sy.Derivative(fx,plane[1] ).doit()
d_fx_c = sy.Derivative(fx,plane[2] ).doit()
d_fx_d = sy.Derivative(fx,plane[3] ).doit()


# Write them to file

# Replace the symbols

In [218]:
str_test = '𝑎(𝑝𝑡𝑦((sin(𝐾)sin(𝑊)+sin(𝑃)cos(𝐾)cos(𝑊))cos(𝑝)cos(𝑤)+(−sin(𝐾)cos(𝑊)+sin(𝑃)sin(𝑊)cos(𝐾))(sin(𝑘)sin(𝑝)cos(𝑤)−sin(𝑤)cos(𝑘))+(sin(𝑘)sin(𝑤)+sin(𝑝)cos(𝑘)cos(𝑤))cos(𝐾)cos(𝑃))+𝑝𝑡𝑧(−(sin(𝐾)sin(𝑊)+sin(𝑃)cos(𝐾)cos(𝑊))sin(𝑤)cos(𝑝)+(−sin(𝐾)cos(𝑊)+sin(𝑃)sin(𝑊)cos(𝐾))(−sin(𝑘)sin(𝑝)sin(𝑤)−cos(𝑘)cos(𝑤))+(sin(𝑘)cos(𝑤)−sin(𝑝)sin(𝑤)cos(𝑘))cos(𝐾)cos(𝑃)))+𝑏(𝑝𝑡𝑦((sin(𝑘)sin(𝑤)+sin(𝑝)cos(𝑘)cos(𝑤))sin(𝐾)cos(𝑃)+(sin(𝐾)sin(𝑃)sin(𝑊)+cos(𝐾)cos(𝑊))(sin(𝑘)sin(𝑝)cos(𝑤)−sin(𝑤)cos(𝑘))+(sin(𝐾)sin(𝑃)cos(𝑊)−sin(𝑊)cos(𝐾))cos(𝑝)cos(𝑤))+𝑝𝑡𝑧((sin(𝑘)cos(𝑤)−sin(𝑝)sin(𝑤)cos(𝑘))sin(𝐾)cos(𝑃)+(sin(𝐾)sin(𝑃)sin(𝑊)+cos(𝐾)cos(𝑊))(−sin(𝑘)sin(𝑝)sin(𝑤)−cos(𝑘)cos(𝑤))−(sin(𝐾)sin(𝑃)cos(𝑊)−sin(𝑊)cos(𝐾))sin(𝑤)cos(𝑝)))+𝑐(𝑝𝑡𝑦((−sin(𝑘)sin(𝑤)−sin(𝑝)cos(𝑘)cos(𝑤))sin(𝑃)+(sin(𝑘)sin(𝑝)cos(𝑤)−sin(𝑤)cos(𝑘))sin(𝑊)cos(𝑃)+cos(𝑃)cos(𝑊)cos(𝑝)cos(𝑤))+𝑝𝑡𝑧((−sin(𝑘)cos(𝑤)+sin(𝑝)sin(𝑤)cos(𝑘))sin(𝑃)+(−sin(𝑘)sin(𝑝)sin(𝑤)−cos(𝑘)cos(𝑤))sin(𝑊)cos(𝑃)−sin(𝑤)cos(𝑃)cos(𝑊)cos(𝑝)))'

In [27]:
def replace_symbs(string):
    
    
#     find_lst = {'cos':'cosd', 'sin':'sind', 
#                 '𝑝𝑡𝑥': 'x_sj', '𝑝𝑡𝑦': 'y_sj', '𝑝𝑡𝑧':'z_sj',
#                 '𝑥':'x_bjg','𝑦':'y_bjg','𝑧':'z_bjg', '𝑤':'w_bjg', '𝑝':'phi_bjg', '𝑘':'K_bjg',
#                 '𝑋':'x_Sjb','𝑌':'y_Sjb','𝑍':'z_Sjb', '𝑊':'w_Sb', '𝑃':'phi_Sb', '𝐾':'K_Sb',
#                 '𝑎':'n_xpg', '𝑏':'n_ypg', '𝑐':'n_zpg', '𝑑':'n_dpg',
#                 '−':'-',
#                '\)cos':')*cos', '\)sin':')*sin',
#                '([^d+-])\(':'\\1*('}


    find_lst = {'cosl':'cosd', 'sinl':'sind', 
                '−':'-'}
    

    new_str = string
    for f,r in find_lst.items():
        new_str = re.sub(f,r,new_str)
    
    return new_str

In [44]:
# print as c code
deriv_func = d_fx_d
code_out = ccode(deriv_func, assign_to="E", type_aliases={real: float80})
code_replaced = replace_symbs(code_out)
code_replaced

'E = 1;'